In [40]:
import json
import pandas as pd

DATA_PATH = '../data/1832_v4_preprocessed.csv'
HIERARCHY_PATH = '../data/circle_packing_data.json'

In [41]:
rec = pd.read_csv(DATA_PATH)
rec.head()

,nom_rue,no_maison,chef_annee_naissance,chef_origine,chef_annee_arrivee,chef_vocation,page,division,chef_vocation_class,chef_origine_class,division_class,division_name
0,marterey,1,1764.0,tolochenaz,1786.0,charon,1,1,NaN,morges,culture,marterey
1,marterey,2,NaN,rolle,1821.0,marechal-ferrant,1,1,artisanat,rolle,culture,marterey
2,marterey,3,1785.0,pailly,NaN,lingere,1,1,service,moudon,culture,marterey
3,marterey,3,NaN,gingins,NaN,rentier,1,1,rente,NaN,culture,marterey
4,marterey,3,NaN,gingins,NaN,rentier,1,1,rente,NaN,culture,marterey


In [42]:
data = {"name": "sup_divisions", "children": []}

In [43]:
rec_sup = rec \
    .groupby(['division_class', 'division_name', 'nom_rue']) \
    .agg({'page': 'count'}) \
    .rename(columns={
        'page': 'value'
        })

rec_sup.head()

value
division_class division_name     nom_rue           
affaires       bourg             bourg          173
                                 st francois     27
               place st francois croix rouge      6
                                 grand chene     62
                                 grotte          28

In [44]:
# Add all children
for sup in rec_sup.index.get_level_values(0).unique():
    # Add empty supdivisions
    data['children'].append({
        "name": sup,
        "children": []
    })

    # Add supdivision children
    for div in rec_sup.loc[sup].index.get_level_values(0).unique():
        data['children'][-1]['children'].append({
            "name": div,
            "children": []
        })
    
        # Add division children
        for rue in rec_sup.loc[sup].loc[div].index:
            data['children'][-1]['children'][-1]['children'].append({
                "name": rue,
                "value": int(rec_sup.loc[sup].loc[div].loc[rue].values[0])
            })

In [45]:
# Save data
with open(HIERARCHY_PATH, 'w') as f:
    json.dump(data, f, indent=4)